# FireMon Policy Optimizer Automation

This notebook automates the process of:
1. Opening FireMon Policy Optimizer
2. Processing first two PO tickets
3. Finding policy revision changes
4. Taking screenshots of specific rule changes

## Stage 1: Setup and Configuration
Install and import required libraries

In [ ]:
# Install required packages
!pip install selenium webdriver-manager pillow

In [ ]:
# Import required libraries
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
import time
import os
from datetime import datetime
import logging

# Setup logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
logger = logging.getLogger(__name__)

In [ ]:
# Configuration - UPDATE THESE VALUES
FIREMON_URL = "https://your-firemon-url.com"  # Replace with your FireMon URL
USERNAME = "your_username"  # Replace with your username
PASSWORD = "your_password"  # Replace with your password

# Create screenshots directory
SCREENSHOT_DIR = "screenshots"
if not os.path.exists(SCREENSHOT_DIR):
    os.makedirs(SCREENSHOT_DIR)
    logger.info(f"Created screenshot directory: {SCREENSHOT_DIR}")

## Stage 2: Initialize WebDriver
Set up Chrome driver with options

In [ ]:
# Setup Chrome options
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--start-maximized')
# Uncomment below to run in headless mode
# chrome_options.add_argument('--headless')

# Initialize Chrome driver
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=chrome_options)
wait = WebDriverWait(driver, 20)  # 20 second timeout for element waits

logger.info("WebDriver initialized successfully")

## Stage 3: Login to FireMon
Navigate to FireMon and perform login

In [ ]:
try:
    # Navigate to FireMon
    logger.info(f"Navigating to FireMon: {FIREMON_URL}")
    driver.get(FIREMON_URL)
    
    # Wait for login page to load
    time.sleep(3)
    
    # Find and fill username field
    username_field = wait.until(EC.presence_of_element_located((By.ID, "username")))
    username_field.clear()
    username_field.send_keys(USERNAME)
    logger.info("Username entered")
    
    # Find and fill password field
    password_field = driver.find_element(By.ID, "password")
    password_field.clear()
    password_field.send_keys(PASSWORD)
    logger.info("Password entered")
    
    # Click login button
    login_button = driver.find_element(By.ID, "login-button")
    login_button.click()
    logger.info("Login button clicked")
    
    # Wait for dashboard to load
    time.sleep(5)
    logger.info("Login successful")
    
except Exception as e:
    logger.error(f"Login failed: {str(e)}")
    raise

## Stage 4: Navigate to Policy Optimizer
Find and click on Policy Optimizer link

In [ ]:
try:
    # Find Policy Optimizer link/menu item
    # Note: Update the selector based on actual FireMon interface
    po_link = wait.until(EC.element_to_be_clickable((By.LINK_TEXT, "Policy Optimizer")))
    po_link.click()
    logger.info("Clicked Policy Optimizer link")
    
    # Wait for PO page to load
    time.sleep(5)
    
    # Wait for PO tickets to load
    po_tickets = wait.until(EC.presence_of_all_elements_located((By.CLASS_NAME, "po-ticket")))
    logger.info(f"Found {len(po_tickets)} PO tickets")
    
except Exception as e:
    logger.error(f"Failed to navigate to Policy Optimizer: {str(e)}")
    raise

## Stage 5: Process First Two PO Tickets
Extract rule names and process each ticket

In [ ]:
# Store ticket information
tickets_to_process = []

try:
    # Get first two PO tickets
    po_tickets = driver.find_elements(By.CLASS_NAME, "po-ticket")[:2]
    
    for i, ticket in enumerate(po_tickets):
        # Extract rule name from ticket
        # Note: Update selector based on actual structure
        rule_name = ticket.find_element(By.CLASS_NAME, "rule-name").text
        
        # Find policy revision link
        revision_link = ticket.find_element(By.CLASS_NAME, "policy-revision-link")
        
        tickets_to_process.append({
            'index': i,
            'rule_name': rule_name,
            'revision_link': revision_link
        })
        
        logger.info(f"Ticket {i+1}: Rule name = {rule_name}")
    
    logger.info(f"Extracted info for {len(tickets_to_process)} tickets")
    
except Exception as e:
    logger.error(f"Failed to extract ticket information: {str(e)}")
    raise

## Stage 6: Process Each Ticket
Click revision link, view changes, and capture screenshots

In [ ]:
# Process each ticket
for ticket_info in tickets_to_process:
    try:
        logger.info(f"\nProcessing ticket {ticket_info['index']+1}: {ticket_info['rule_name']}")
        
        # Store current window handle
        main_window = driver.current_window_handle
        
        # Click on policy revision link
        ticket_info['revision_link'].click()
        logger.info("Clicked policy revision link")
        
        # Switch to new window/tab
        time.sleep(2)
        for window_handle in driver.window_handles:
            if window_handle != main_window:
                driver.switch_to.window(window_handle)
                break
        
        # Wait for page to load
        time.sleep(3)
        
        # Find and click "View Changes" toggle button
        view_changes_btn = wait.until(EC.element_to_be_clickable((By.XPATH, "//button[contains(text(), 'View Changes')]|//button[contains(text(), 'view changes')]|//button[@class='view-changes-toggle']|//button[@id='view-changes']")))
        view_changes_btn.click()
        logger.info("Clicked 'View Changes' button")
        
        # Wait for changes to load
        time.sleep(5)
        
        # Find all rule change rows
        rule_rows = driver.find_elements(By.CLASS_NAME, "rule-change-row")
        logger.info(f"Found {len(rule_rows)} rule change rows")
        
        # Find the specific rule matching our rule name
        found_matching_rule = False
        for row in rule_rows:
            try:
                # Get rule name from row (update selector as needed)
                row_rule_name = row.find_element(By.CLASS_NAME, "rule-name").text
                
                if row_rule_name == ticket_info['rule_name']:
                    logger.info(f"Found matching rule: {row_rule_name}")
                    
                    # Scroll to element
                    driver.execute_script("arguments[0].scrollIntoView(true);", row)
                    time.sleep(1)
                    
                    # Take screenshot
                    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
                    screenshot_name = f"{SCREENSHOT_DIR}/PO_ticket_{ticket_info['index']+1}_{ticket_info['rule_name']}_{timestamp}.png"
                    row.screenshot(screenshot_name)
                    logger.info(f"Screenshot saved: {screenshot_name}")
                    
                    found_matching_rule = True
                    break
                    
            except Exception as e:
                logger.warning(f"Error processing row: {str(e)}")
                continue
        
        if not found_matching_rule:
            logger.warning(f"No matching rule found for: {ticket_info['rule_name']}")
            # Take full page screenshot as fallback
            timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
            screenshot_name = f"{SCREENSHOT_DIR}/PO_ticket_{ticket_info['index']+1}_full_page_{timestamp}.png"
            driver.save_screenshot(screenshot_name)
            logger.info(f"Full page screenshot saved: {screenshot_name}")
        
        # Close current tab and switch back to main window
        driver.close()
        driver.switch_to.window(main_window)
        time.sleep(2)
        
    except Exception as e:
        logger.error(f"Error processing ticket {ticket_info['index']+1}: {str(e)}")
        # Try to switch back to main window
        try:
            driver.switch_to.window(main_window)
        except:
            pass

## Stage 7: Cleanup
Close browser and summarize results

In [ ]:
# Print summary
logger.info("\n=== AUTOMATION COMPLETE ===")
logger.info(f"Screenshots saved in: {SCREENSHOT_DIR}")

# List all screenshots
screenshots = [f for f in os.listdir(SCREENSHOT_DIR) if f.endswith('.png')]
logger.info(f"Total screenshots captured: {len(screenshots)}")
for screenshot in screenshots:
    logger.info(f"  - {screenshot}")

In [ ]:
# Close browser
driver.quit()
logger.info("Browser closed")

## Troubleshooting Tips

1. **Login Issues:**
   - Check if username/password field IDs are correct
   - Verify credentials are correct
   - Check if there's a captcha or 2FA

2. **Element Not Found:**
   - Use browser developer tools to inspect elements
   - Update selectors (ID, class name, XPath) accordingly
   - Increase wait times if elements load slowly

3. **Screenshot Issues:**
   - Ensure screenshot directory has write permissions
   - Check if element is visible before taking screenshot
   - Use full page screenshot as fallback

4. **Common Selectors to Update:**
   - Login: `#username`, `#password`, `#login-button`
   - PO Tickets: `.po-ticket`, `.rule-name`, `.policy-revision-link`
   - Changes: `.view-changes-toggle`, `.rule-change-row`